<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: Calwebb_Detector1, RSCD step

<span style="color:red"> **Instruments Affected**</span>: MIRI

Tested on MIRI Simulated data

### Table of Contents
<div style="text-align: left"> 

<br>  [Introduction](#intro_ID) <br> [Imports](#imports_ID) <br> [Run Pipeline Steps](#pipeline_ID) <br> [Check Flagged Frames](#flagging_ID) <br> [Results](#results_ID) <br> [About This Notebook](#about_ID) <br>


</div>

<a id="intro_ID"></a>
# Introduction


This test is designed to test the RSCD step in the calwebb_detector1 pipeline. This step only applies to MIRI. The Reset Switch Charge Decay (RSCD) step corrects for a set of ramp effects that affect all ingrations after the first by simply flagging the first N groups as DO_NOT_USE. The number of groups to flag is read from the reference file based on readout mode and subarray. For more information on the pipeline step visit the links below. 

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/rscd/description.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/rscd

Most of the data for this test were created with the MIRI Data Simulator, and the documentation for that code can be found here: http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public. This test can also be run on properly formatted ground test data.


### Calibration WG Requested Algorithm: 

A short description and link to the page: https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Reset+Switch+Charge+Decay+Correction


### Defining Terms
Definition of terms or acronymns.

JWST: James Webb Space Telescope

MIRI: Mid-Infrared Instrument

MIRISim: MIRI Data Simulator

RSCD: Reset Switch Charge Decay

### Description of test

This test is performed by reading in data of various types (subarray, readpattern, etc.) and running them through the rscd step to see whether the correct number of frames are flagged (read from the reference file).

### Create the data for testing

The set of data used in this particular test were created with the MIRI Data Simulator (MIRISim). Referring to the MIRISim link, you can see how to set up and run the simulator to re-create the input files if you wish. The data was run with a scene.ini file that specified what the scene should look like, with coordinates for the stars given in units of arcsecond offsets from the center of the field of view. The scene.ini file as well as the setup files simuation.ini and simulator.ini are needed to run the simulation.

Once in the mirisim conda environment, the simulation is run with the command line:
> mirisim simulation.ini




[Top of Page](#title_ID)

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /grp/crds/cache


<a id="imports_ID"></a>
# Imports
The following packages will need to be imported for the scripts to work.

* os for reading in CRDS server info and path information
* ci_watson and get_bigdata for reading data from artifactory
* jwst.datamodels for opening files as a JWST Datamodel
* jwst.pipeline to run the pipeline step/module
* jwst.associations to create association table
* numpy for calculations
* inspect and Markdown to get documentation for step being tested  
* photutils for star finding and aperture photometry


[Top of Page](#title_ID)

In [3]:
import os
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
import crds

from astropy.io import fits
import numpy as np
from IPython.display import Markdown
from jwst.datamodels import RampModel, dqflags
from jwst.dq_init import DQInitStep
from jwst.rscd import RscdStep
from ci_watson.artifactory_helpers import get_bigdata
import inspect
from IPython.display import Markdown

In [4]:
# Get raw python docstring to see documentation for this step
raw = inspect.getdoc(RscdStep) or ''

# To convert to markdown, you need convert line breaks from \n to <br />
markdown_text = "<br />".join(raw.split("\n"))

# Here you can format markdown as an output using the Markdown method.
Markdown("""
# rscd_step
---
{}
""".format(markdown_text))


# rscd_step
---
RscdStep: Performs an RSCD correction to MIRI data.<br />Baseline version flags the first N groups as 'DO_NOT_USE' in<br />the 2nd and later integrations in a copy of the input<br />science data model.<br />Enhanced version is not ready nor enabled.


<a id="pipeline_ID"></a>
### Run Pipeline Steps

For this test, run each file through dq_init and the rscd steps only. The output for the test is the output of the rscd step. We are testing that the specified number of frames for each file is flagged properly.

[Top of Page](#title_ID)

In [5]:
# Put in tests for FULL frame fast, FULL frame slow, subarray and FASTGRPAVG (should skip FASTGRPAVG) 

# read in input files

file1 = get_bigdata('jwst_validation_notebooks',
                    'validation_data',
                    'rscd',
                    'rscd_miri_test', 
                    'starfield_slow_seq1_MIRIMAGE_F1130Wexp1.fits')

file2 = get_bigdata('jwst_validation_notebooks',
                    'validation_data',
                    'rscd',
                    'rscd_miri_test', 
                    'starfield_fast_4ptdither_seq1_MIRIMAGE_F1130Wexp1.fits')

file3 = get_bigdata('jwst_validation_notebooks',
                    'validation_data',
                    'rscd',
                    'rscd_miri_test', 
                    'det_image_seq1_MIRIMAGE_F1130Wexp1_brightsky.fits')

file4 = get_bigdata('jwst_validation_notebooks',
                    'validation_data',
                    'rscd',
                    'rscd_miri_test', 
                    'starfield_fastgrpavg_seq1_MIRIMAGE_F1130Wexp1.fits')

file5 = get_bigdata('jwst_validation_notebooks',
                    'validation_data',
                    'rscd',
                    'rscd_miri_test', 
                    'jw04193001001_01101_00001_MIRIFULONG_uncal.fits')

filelist = get_bigdata('jwst_validation_notebooks',
                       'validation_data',
                       'rscd',
                       'rscd_miri_test', 
                       'rscd_testfiles.txt')

rscdoutfiles = 'rscd_processed_files.txt'

# read in list of input files
imlist1 = np.loadtxt(filelist,dtype=bytes, ndmin=1).astype(str)  # avoid the 'b at the start of a line

# open output file for ramp fit file names
f = open(rscdoutfiles,'w')

print('There are ', imlist1.size, ' images.')

# loop over list of files
for file in imlist1:
       
    # set up output file name
    if 'uncal' in file:
        base, remainder = file.split('_uncal')
    else:    
        base, remainder = file.split('.')
    # write out name of output files to text file for later use
    f.write(base+'_rscdstep.fits'+'\n')
    
    # check that header has needed keywords set
    model = RampModel(file)
    preim = DQInitStep.call(model)
    postim = RscdStep.call(preim, output_file=base)
    
f.close()

print('All files processed.')

There are  5  images.


2022-03-24 13:57:01,595 - stpipe.DQInitStep - INFO - DQInitStep instance created.


2022-03-24 13:57:01,669 - stpipe.DQInitStep - INFO - Step DQInitStep running with args (<RampModel(3, 10, 1024, 1032) from starfield_slow_seq1_MIRIMAGE_F1130Wexp1.fits>,).


2022-03-24 13:57:01,670 - stpipe.DQInitStep - INFO - Step DQInitStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}


2022-03-24 13:57:01,694 - stpipe.DQInitStep - INFO - Using MASK reference file /grp/crds/cache/references/jwst/jwst_miri_mask_0028.fits


...

2022-03-24 13:57:08,498 - stpipe.RscdStep - INFO - Step RscdStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpxfebrclz/jw04193001001_01101_00001_MIRIFULONG', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'type': 'baseline'}


2022-03-24 13:57:08,520 - stpipe.RscdStep - INFO - Using RSCD reference file /grp/crds/cache/references/jwst/jwst_miri_rscd_0015.fits


2022-03-24 13:57:09,544 - stpipe.RscdStep - INFO - Saved model in jw04193001001_01101_00001_MIRIFULONG_rscdstep.fits


2022-03-24 13:57:09,545 - stpipe.RscdStep - INFO - Step RscdStep done


All files processed.


<a id="flagging_ID"></a>

### Check flagged frames

Check that the frames at the beginning of each integration after the first are flagged properly.

FAST mode should mark 4 frames as 'DO_NOT"USE', SLOW mode marks 2 frames, and step should skip FASTGRPAVG.
All subarrays are set to skip 4 frames. (These are the values currently set in the reference files as of 8/2020.)

[Top of Page](#title_ID)

In [6]:
# FAST mode should mark 4 frames as 'DO_NOT"USE', SLOW mode marks 2 frames, and step should skip FASTGRPAVG.
# All subarrays are set to skip 4 frames.

# read in list of input files
imlist2 = np.loadtxt(rscdoutfiles,dtype=bytes, ndmin=1).astype(str)  # avoid the 'b at the start of a line

# loop over list of files
for rscdfile in imlist2:
    # Read in RSCD output file

    rscdmodel = RampModel(rscdfile)
    
    # access groupdq extension of the file
    groupdq = rscdmodel.groupdq
    
    # Get readout pattern from science header
    readpatt = rscdmodel.meta.exposure.readpatt
    science_subarray = rscdmodel.meta.subarray.name
    
    # Get reference file used in step
    rscdreffile = rscdmodel.meta.ref_file.rscd.name
    print('RSCD reference file', rscdreffile)
    
    # parse linfile name to discard crds://
    file = rscdreffile.split("/")[2]
    
    # read in reference file
    # find location of file
    basename = crds.core.config.pop_crds_uri(rscdreffile)
    path = crds.locate_file(basename, "jwst")
 
    # open reference file
    reffile = fits.open(path)
    reftable = reffile['RSCD_GROUP_SKIP'].data
    refhead = reffile[0].header
    
    # read in table value for number to skip from reference file
    mask = ((reftable['READPATT'] == readpatt) & (reftable['SUBARRAY'] == 'FULL'))
    row=reftable[mask]
    numskiparray = row['group_skip']
    
    if numskiparray.size > 0:
        numskip = row['group_skip'][0] 
    else:
        numskip = 0
        
    # For all integrations after the first, flag numskip frames at the start of the ramp as 'DO_NOT_USE'
    nint = rscdmodel.meta.exposure.nints
 
    # Use assert statement to check that all integrations greater than the first and in cases where 
    # the number of frames to skip is not 0, check that all groups are properly flagged.
    for int in range(nint):
        if (int > 0 and numskip > 0):
            assert np.all(groupdq[int, 0:numskip-1, 400, 400] == dqflags.group['DO_NOT_USE'])
    
    # output print to screen to see output of tests
    print('Science file tested: ',rscdfile)
    print('Subarray: ', science_subarray, ' Readout pattern: ', readpatt, ' Number of frames to flag: ', numskip)
    print('Data quality flags for each frame in each integration')
    print(groupdq[:,:,400,400])
    print()

print('All files processed. If no Assertion Errors printed, all tests passed.')    

RSCD reference file crds://jwst_miri_rscd_0014.fits
Science file tested:  starfield_slow_seq1_MIRIMAGE_F1130Wexp1_rscdstep.fits
Subarray:  FULL  Readout pattern:  SLOW  Number of frames to flag:  2
Data quality flags for each frame in each integration
[[0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]]



RSCD reference file crds://jwst_miri_rscd_0014.fits
Science file tested:  starfield_fast_4ptdither_seq1_MIRIMAGE_F1130Wexp1_rscdstep.fits
Subarray:  FULL  Readout pattern:  FAST  Number of frames to flag:  4
Data quality flags for each frame in each integration
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

RSCD reference file crds://jwst_miri_rscd_0014.fits
Science file tested:  det_image_seq1_MIRIMAGE_F1130Wexp1_brightsky_rscdstep.fits
Subarray:  BRIGHTSKY  Readout pattern:  FAST  Number of frames to flag:  4
Data quality flags for each frame in each integration
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]



RSCD reference file crds://jwst_miri_rscd_0014.fits
Science file tested:  starfield_fastgrpavg_seq1_MIRIMAGE_F1130Wexp1_rscdstep.fits
Subarray:  FULL  Readout pattern:  FASTGRPAVG  Number of frames to flag:  0
Data quality flags for each frame in each integration
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]



RSCD reference file crds://jwst_miri_rscd_0015.fits
Science file tested:  jw04193001001_01101_00001_MIRIFULONG_rscdstep.fits
Subarray:  FULL  Readout pattern:  FAST  Number of frames to flag:  4
Data quality flags for each frame in each integration
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

All files processed. If no Assertion Errors printed, all tests passed.


<a id="results_ID"></a>
### Results: Pass/Fail criteria

As long as there are no Assertion Errors, and the number shown along with the subarray and Readout pattern matches the number of frames flagged at the start of the integration for all integrations after the first, then the test passes.

For example:

FULL    FAST    4

[0 0 0 0 0 0 0 0 0 0]

[1 1 1 1 0 0 0 0 0 0]

This indicates a passing test.

[Top of Page](#title_ID)

<a id="about_ID"></a>
## About this Notebook
**Author:** Misty Cracraft, Senior Staff Scientist, MIRI Branch
<br>**Updated On:** 08/03/2020